# DHd Affiliations

aim: parse XML file, generate a CSV for manual refinement in Open Refine

In [1]:
!ls ..

README.md                          affiliations_distinct_distinct.xml
affiliations_alle.xml              scripts
affiliations_distinct.xml


In [2]:
xml_source_file_name = "affiliations_distinct_distinct.xml"
xml_source_file_path = "../" + xml_source_file_name 
xml_source_file_path

'../affiliations_distinct_distinct.xml'

In [3]:
import xml.etree.ElementTree as ET
xml = ET.parse(xml_source_file_path)
xml = xml.getroot()

affiliation_elems = xml.findall("affiliation")
len(affiliation_elems)

978

In [4]:
affiliation_elems[0].text

'Austrian Centre for Digital Humanities and Cultural Heritage'

In [5]:
def get_country_rule_based(value:str):
    """
    Rule-based detection of country by simple string matching
    """
    if value != None:
        
        #try rule-based detection of country
        if "Österreich" in value:
            return "Österreich"
        elif "Austria" in value:
            return "Österreich"
        elif "Deutschland" in value:
            return "Deutschland"
        elif "Germany" in value:
            return "Deutschland"
        elif "Schweiz" in value:
            return "Schweiz"
        elif "Switzerland" in value:
            return "Schweiz"
        
        #found some other countries in the GPE detected by NLTK and updated the function
        elif "Greece" in value:
            return "Griechenland"
        elif "Griechenland" in value:
            return "Griechenland"
        elif "United Kingdom" in value:
            return "Vereinigtes Königreich"
        elif "Vereinigtes Königreich" in value:
            return "Vereinigtes Königreich"
        elif "Indien" in value:
            return "Indien"
        elif "Italien" in value:
            return "Italien"
        elif "Russland" in value:
            return "Russland"
        elif "Russia" in value:
            return "Russland"
        elif "Canada" in value:
            return "Kanada"
        elif "Norwegen" in value:
            return "Norwegen"
        elif "Norway" in value:
            return "Norwegen"
        elif "Australien" in value:
            return "Australien"
        elif "Serbien" in value:
            return "Serbien"
        elif "Estland" in value:
            return "Estland"
        elif "Niederlande" in value:
            return "Niederlande"
        elif "Israel" in value:
            return "Israel"
        elif "Irland" in value:
            return "Irland"
        elif "Ireland" in value:
            return "Irland"
        elif "Spain" in value:
            return "Spanien"
        elif "Spanien" in value:
            return "Spanien"
        elif "Portugal" in value:
            return "Portugal"
        elif "Belgien" in value:
            return "Belgien"
        elif "France" in value:
            return "Frankreich"
        elif "Frankreich" in value:
            return "Frankreich"
        elif "China" in value:
            return "China"
        elif "Taiwan" in value:
            return "Taiwan"
        
    
        else: 
            return None
    else:
        return None
    
    

In [ ]:
!python -m spacy download xx_ent_wiki_sm

In [6]:
import spacy
NER = spacy.load("xx_ent_wiki_sm")
def ner_spacy(value:str):
    ents = NER(value)
    #for ent in ents.ents:
    #print(ent.text,ent.label_)
    result = []
    for ent in ents.ents:
        result_item = {}
        result_item["text"] = ent.text
        result_item["type"] = ent.label_
        result.append(result_item)
    
    return result

In [7]:
import nltk
nltk.download('maxent_ne_chunker')
nltk.download('punkt')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/ingoboerner/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ingoboerner/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/ingoboerner/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ingoboerner/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:
def ner_nltk(value:str):
    result= []
    for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(value))):
        if hasattr(chunk, 'label'):
            #print(chunk.label(), ' '.join(c[0] for c in chunk))
            if chunk.label() == "GPE":
                chunk_text = ' '.join(c[0] for c in chunk)
                #print(chunk_text)
                result.append(chunk_text)
    return result

In [ ]:
ner_nltk("École polytechnique fédérale de Lausanne")

In [9]:
import re

In [10]:
affiliations = []
for affiliation_elem in affiliation_elems:
    affiliation = {}
    if affiliation_elem.text != None:
        orig_string = affiliation_elem.text.strip().replace("\n","")
        orig_string = re.sub(' +', ' ', orig_string)
        affiliation["orig_string"] = orig_string
        #try simple rule-based detection of country
        affiliation["country"] = get_country_rule_based(orig_string)
    
        # NER
        affiliation["spacy_entities"] = ner_spacy(orig_string)
        affiliation["nltk_gpe"] = ner_nltk(orig_string)
    
    # add enriched to array 
    affiliations.append(affiliation)


In [11]:
affiliations[220]

{'orig_string': 'The University of Edinburgh, Law School, Vereinigtes Königreich',
 'country': 'Vereinigtes Königreich',
 'spacy_entities': [{'text': 'University of Edinburgh', 'type': 'ORG'}],
 'nltk_gpe': ['Edinburgh']}

In [ ]:
#create and clean a list of gpe detected by nltk
nltk_gpe_vals = []
for aff in affiliations:
    if "nltk_gpe" in aff:
        for gpe in aff["nltk_gpe"]:
            if gpe not in nltk_gpe_vals:
                nltk_gpe_vals.append(gpe)

In [ ]:
nltk_gpe_vals

In [ ]:
import json
jsonString = json.dumps(nltk_gpe_vals)
jsonFile = open("nltk_gpe_vals.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

Manually cleaning this file, should be loaded again and used to detect cities. When cleaning I also found some additional countries, that were added to the rule-based function above; will be run again.

In [12]:
affiliations[415]

{'orig_string': 'Centar za digitalne humanističke nauke, Serbien',
 'country': 'Serbien',
 'spacy_entities': [],
 'nltk_gpe': ['Serbien']}

In [13]:
affiliations[0]

{'orig_string': 'Austrian Centre for Digital Humanities and Cultural Heritage',
 'country': 'Österreich',
 'spacy_entities': [{'text': 'Austrian Centre for Digital Humanities',
   'type': 'ORG'}],
 'nltk_gpe': ['Austrian']}

Cities, that where detected by nltk as GPE and cleaned manually.

In [19]:
import json
with open('nltk_gpe_vals.json') as f:
    cities = json.load(f)

In [20]:
cities

['Moskau',
 'Hagen',
 'Bochum',
 'Graz',
 'Köln',
 'Trier',
 'Zurich',
 'Freiburg',
 'Rom',
 'Mainz',
 'Leipzig',
 'Halle',
 'Kraków',
 'Edinburgh',
 'Passau',
 'Basel',
 'Bern',
 'Saarbrücken',
 'Passau',
 'Wien',
 'Hamburg',
 'Berlin',
 'Roma',
 'Frankfurt',
 'Paris',
 'Victoria',
 'Mannheim',
 'Oslo',
 'Illinois',
 'Bonn',
 'Lonfon',
 'New Mexico',
 'Jena',
 'Amsterdam',
 'Basel',
 'Cambridge',
 'Sheffield',
 'Verona',
 'Italy',
 'Wien',
 'Lausanne',
 'Umea',
 'Jena',
 'Beijing',
 'Salzburg',
 'Semedia',
 'Ancona',
 'Konstanz',
 'València',
 'Bergen',
 'Kassel']

In [22]:
for aff in affiliations:
    if 'orig_string' in aff:
        for city in cities:
            if city in aff['orig_string']:
                aff["city"] = city

In [26]:
affiliations[3]

{'orig_string': 'Berlin-Brandenburgische Akademie der Wissenschaften',
 'country': None,
 'spacy_entities': [{'text': 'Berlin-Brandenburgische Akademie der Wissenschaften',
   'type': 'ORG'}],
 'nltk_gpe': [],
 'city': 'Berlin'}

In [27]:
affiliations[220]

{'orig_string': 'The University of Edinburgh, Law School, Vereinigtes Königreich',
 'country': 'Vereinigtes Königreich',
 'spacy_entities': [{'text': 'University of Edinburgh', 'type': 'ORG'}],
 'nltk_gpe': ['Edinburgh'],
 'city': 'Edinburgh'}

In [28]:
affiliations[221]

{'orig_string': 'TU Berlin, Deutschland',
 'country': 'Deutschland',
 'spacy_entities': [{'text': 'TU Berlin', 'type': 'ORG'},
  {'text': 'Deutschland', 'type': 'LOC'}],
 'nltk_gpe': ['Deutschland'],
 'city': 'Berlin'}

In [29]:
affiliations[222]

{'orig_string': 'Bibliotheca Hertziana, Rom, Italien',
 'country': 'Italien',
 'spacy_entities': [{'text': 'Bibliotheca Hertziana', 'type': 'LOC'}],
 'nltk_gpe': ['Rom', 'Italien'],
 'city': 'Rom'}

In [32]:
affiliations[225]

{'orig_string': 'TU Dresden, Deutschland',
 'country': 'Deutschland',
 'spacy_entities': [{'text': 'TU Dresden', 'type': 'ORG'},
  {'text': 'Deutschland', 'type': 'LOC'}],
 'nltk_gpe': ['Deutschland']}

In [33]:
len(affiliations)

978

Write a csv and clean in Open Refine

In [34]:
import csv

In [37]:
import csv

header = ['orig_string', 'country', 'city']

with open('affiliations_openrefine.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for aff in affiliations:
        row = []
        for col in header:
            if col in aff:
                row.append(aff[col])
            else:
                row.append("")
        writer.writerow(row)